# Average Annual Household Transportation Costs
Out-of-pocket expenses including tolls, parking, and transit fares

These costs do not include other vehicle operating and purchase costs.

Costs are annualized using factor of 300, as used in the 2018 RTP.


In [1]:
import os, sys, shutil
sys.path.append(os.path.join(os.getcwd(),"inputs"))
sys.path.append(os.path.join(os.getcwd(),r"../../../scripts/summarize/standard"))
sys.path.append(os.path.join(os.getcwd(),r'../../..'))
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import sqlalchemy
from shapely import wkt
from input_configuration import *
from emme_configuration import *
from standard_summary_configuration import *
pd.options.mode.chained_assignment = None
import os, sys, time, shutil
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
pd.options.mode.chained_assignment = None
import h5py

%matplotlib inline

from IPython.display import display, HTML

if (sys.version_info < (3, 0)):
    HTML('''<script>
    code_show=true; 
    function code_toggle() {
     if (code_show){
     $('div.input').hide();
     } else {
     $('div.input').show();
     }
     code_show = !code_show
    } 
    $( document ).ready(code_toggle);
    </script>
    <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

# Relative path between notebooks and goruped output directories
relative_path = '../../../outputs/agg/dash'
survey_path = '../../../inputs/base_year/survey'

In [3]:
df = pd.read_csv(r'../../../outputs/agg/dash/person_cost.csv')
df_hh = pd.read_csv(r'..\..\..\outputs\agg\dash\hh_geog.csv')

annual_factor = 300

df['is_rgc'] = 0
df.loc[df['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1
df_hh['is_rgc'] = 0
df_hh.loc[df_hh['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1

In [4]:
df = df[(((df['mode'].isin(['SOV','HOV2','HOV3+']) & (df['dorp'] == 1)) 
          | (~df['mode'].isin(['SOV','HOV2','HOV3+']))))]

In [5]:
pd.options.display.float_format = '${:0,.0f}'.format

# County

In [6]:
_df = pd.DataFrame(df.groupby('hh_county').sum()['travcost_wt']/df_hh.groupby('hh_county').sum()['hhexpfac'])*annual_factor
_df.columns = ['Average Annual Costs']
_df.loc['Region'] = df['travcost_wt'].sum()/df_hh['hhexpfac'].sum()*annual_factor
_df

,Average Annual Costs
hh_county,
King,"$3,490"
Kitsap,"$3,692"
Pierce,"$3,808"
Snohomish,"$4,306"
Region,"$3,727"


# Regional Growth Center

In [7]:
_df = pd.DataFrame(df.groupby('is_rgc').sum()['travcost_wt']/df_hh.groupby('is_rgc').sum()['hhexpfac'])*annual_factor
_df.columns = ['Average Annual Costs']
_df

,Average Annual Costs
is_rgc,
0,"$4,172"
1,"$1,715"


In [8]:
_df = pd.DataFrame(df.groupby('hh_rgc').sum()['travcost_wt']/df_hh.groupby('hh_rgc').sum()['hhexpfac'])*annual_factor
_df.columns = ['Average Annual Costs']
_df

,Average Annual Costs
hh_rgc,
Auburn,"$2,304"
Ballard-Interbay,"$2,181"
Bellevue,"$1,412"
Bothell Canyon Park,"$4,039"
Bremerton,"$1,298"
Burien,"$2,513"
Cascade,"$2,895"
Duwamish,"$2,749"
Everett,"$1,165"


# Regional Geography

In [9]:
_df = pd.DataFrame(df.groupby('hh_rg_proposed').sum()['travcost_wt']/df_hh.groupby('hh_rg_proposed').sum()['hhexpfac'])*annual_factor
_df.columns = ['Average Annual Costs']
_df

,Average Annual Costs
hh_rg_proposed,
CitiesTowns,"$5,211"
Core,"$3,550"
HCT,"$4,143"
Metro,"$2,461"
Rural,"$6,620"
UU,"$5,007"


# Equity Geography

In [10]:
results_df = pd.DataFrame()
trip_rate_df = pd.DataFrame()
list_50 = ['hh_racial','hh_poverty']

# Calculate total work trips per each group
work_results_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        if df_hh[df_hh[equity_geog+geog_type] == 1].sum()['hhexpfac'] > 0:
            _df = (df[df[equity_geog+geog_type] == 1].sum()['travcost_wt']/df_hh[df_hh[equity_geog+geog_type] == 1].sum()['hhexpfac'])*annual_factor
            results_df.loc[equity_geog+geog_type,'costs'] = _df
        
inv_results_df = pd.DataFrame()
trip_rate_df = pd.DataFrame()
list_50 = ['hh_racial','hh_poverty']

# Calculate total work trips per each group
work_results_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        if df_hh[df_hh[equity_geog+geog_type] == 1].sum()['hhexpfac'] > 0:
            _df = (df[df[equity_geog+geog_type] == 0].sum()['travcost_wt']/df_hh[df_hh[equity_geog+geog_type] == 0].sum()['hhexpfac'])*annual_factor
            inv_results_df.loc[equity_geog+geog_type,'costs'] = _df
        
# inv_results_df.index = ['NOT in '+i for i in results_df.index] 
_df = results_df.merge(inv_results_df, left_index=True, right_index=True)
_df.columns = ['Inside Equity Geog','Outside Equity Geog']
_df[~_df.isnull()]

,Inside Equity Geog,Outside Equity Geog
hh_youth_reg,"$4,460","$3,098"
hh_elderly_reg,"$3,857","$3,616"
hh_elderly_50,"$5,685","$3,725"
hh_english_reg,"$3,500","$3,874"
hh_racial_reg,"$3,240","$4,183"
hh_racial_50,"$3,405","$3,814"
hh_poverty_reg,"$3,211","$4,156"
hh_poverty_50,"$2,659","$3,757"
hh_disability_reg,"$3,502","$3,941"
